In [1]:
import keras # imports keras and tensorflow as backend
import matplotlib.pyplot as plt # imports matplotlib
import numpy as np
import pandas as pd
%matplotlib inline
import os
import time

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
NAME = "DeepRecommender-Sequential-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir='./logs/{}'.format(NAME))

In [3]:
# Data Paths
DATA_DIR = './nf_data'

nf_3m_train = os.path.join(DATA_DIR, 'N3M_TRAIN', 'n3m.train.txt')
nf_3m_valid = os.path.join(DATA_DIR, 'N3M_VALID', 'n3m.valid.txt')
nf_3m_test = os.path.join(DATA_DIR, 'N3M_TEST', 'n3m.test.txt')

In [4]:
df_train = pd.read_csv(nf_3m_train, names=['CustomerID','MovieID','Rating'], sep='\t')
print(df_train.shape)
df_train.head()

(13675402, 3)


,CustomerID,MovieID,Rating
0,0,4884,4.0
1,0,5332,4.0
2,0,5633,4.0
3,0,6642,4.0
4,0,6922,4.0


In [5]:
df_train.iloc[:,1][(df_train.iloc[:,0]==0) & (df_train.iloc[:,1]<=1000)]

30    829
42    159
43    904
60     30
61    679
72    859
Name: MovieID, dtype: int64

In [6]:
df_valid = pd.read_csv(nf_3m_valid, names=['CustomerID','MovieID','Rating'], sep='\t')
print(df_valid.shape)
df_valid.head()

(1042131, 3)


,CustomerID,MovieID,Rating
0,0,1012,4.0
1,0,2977,4.0
2,0,4101,4.0
3,0,15858,4.0
4,0,16629,4.0


In [7]:
df_test = pd.read_csv(nf_3m_test, names=['CustomerID','MovieID','Rating'], sep='\t')
print(df_test.shape)
df_test.head()

(1040428, 3)


,CustomerID,MovieID,Rating
0,0,9529,5.0
1,0,3363,4.0
2,0,5837,4.0
3,0,6017,4.0
4,0,10476,4.0


In [8]:
customer_map = df_train.CustomerID.unique()
customer_map.sort()
customer_map = customer_map[0:17550]
#user_map
num_users = len(customer_map)
customer_map[-1]

22203

In [9]:
movie_map = df_train.MovieID.unique()
movie_map.sort()
movie_map = movie_map[0:1000]
#movie_map
num_movies = len(movie_map)
movie_map[-1]

1000

In [10]:
# file = open('matrix_train.csv','w')

# def temp(i):
#     return np.where(movie_map == i)[0][0]


# for id_user in customer_map:
#     id_movie = df_train.iloc[:,1][(df_train.iloc[:,0]==id_user) & (df_train.iloc[:,1]<=1000)]
#     id_movie = id_movie.apply(temp)
#     id_rating=df_train.iloc[:,2][(df_train.iloc[:,0]==id_user) & (df_train.iloc[:,1]<=1000)]
#     ratings=np.zeros(num_movies, dtype=np.uint32)
#     ratings[id_movie-1]=id_rating
#     if sum(ratings)==0:
#         continue
#     ratings=pd.DataFrame(ratings.reshape(-1, len(ratings)))
#     file.write(ratings.to_csv(index=False, header=False))
#     del id_movie
#     del id_rating
#     del ratings


# file.close()


In [11]:
X = pd.read_csv('matrix_train.csv', header=None)
print(X.shape)
X.head()

(13118, 1000)


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
X.shape[0]

13118

In [13]:
customer_map = df_valid.CustomerID.unique()
customer_map.sort()
customer_map = customer_map[:7020]
num_users = len(customer_map)
customer_map[-1]

14710

In [14]:
movie_map = df_valid.MovieID.unique()
movie_map.sort()
movie_map = movie_map[:895]
#movie_map
num_movies = len(movie_map)
movie_map[-1]

1000

In [15]:
# file = open('matrix_valid.csv','w')

# def temp(i):
#     return np.where(movie_map == i)[0][0]


# for id_user in customer_map:
#     id_movie = df_valid.iloc[:,1][(df_valid.iloc[:,0]==id_user) & (df_valid.iloc[:,1]<=1000)]
#     id_movie = id_movie.apply(temp)
#     id_rating=df_valid.iloc[:,2][(df_valid.iloc[:,0]==id_user) & (df_valid.iloc[:,1]<=1000)]
#     ratings=np.zeros(1000, dtype=np.uint32)
#     ratings[id_movie-1]=id_rating
#     if sum(ratings)==0:
#         continue
#     ratings=pd.DataFrame(ratings.reshape(-1, len(ratings)))
#     file.write(ratings.to_csv(index=False, header=False))
#     del id_movie
#     del id_rating
#     del ratings


# file.close()

In [16]:
X_valid = pd.read_csv('matrix_valid.csv', header=None)
print(X_valid.shape)
X_valid.head()

(2255, 1000)


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X_valid.shape[0]

2255

In [18]:
customer_map = df_test.CustomerID.unique()
customer_map.sort()
customer_map = customer_map[:7020]
num_users = len(customer_map)
customer_map[-1]

14678

In [19]:
movie_map = df_test.MovieID.unique()
movie_map.sort()
movie_map = movie_map[:901]
#movie_map
num_movies = len(movie_map)
movie_map[-1]

1000

In [20]:
# file = open('matrix_test.csv','w')

# def temp(i):
#     return np.where(movie_map == i)[0][0]


# for id_user in customer_map:
#     id_movie = df_test.iloc[:,1][(df_test.iloc[:,0]==id_user) & (df_test.iloc[:,1]<=1000)]
#     id_movie = id_movie.apply(temp)
#     id_rating=df_test.iloc[:,2][(df_test.iloc[:,0]==id_user) & (df_test.iloc[:,1]<=1000)]
#     ratings=np.zeros(1000, dtype=np.uint32)
#     ratings[id_movie-1]=id_rating
#     if sum(ratings)==0:
#         continue
#     ratings=pd.DataFrame(ratings.reshape(-1, len(ratings)))
#     file.write(ratings.to_csv(index=False, header=False))
#     del id_movie
#     del id_rating
#     del ratings


# file.close()

In [21]:
X_test = pd.read_csv('matrix_test.csv', header=None)
print(X_test.shape)
X_test.head()

(2286, 1000)


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
X_test.shape[0]

2286

In [23]:
import keras.backend as K

def rmse(y_true, y_pred):
    mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
    masked_squared_error = mask_true * K.square((y_true - y_pred))
    # in case mask_true is 0 everywhere, the error would be nan, therefore divide by at least 1
    # this doesn't change anything as where sum(mask_true)==0, sum(masked_squared_error)==0 as well
    masked_mse = K.sum(masked_squared_error, axis=-1) / K.sum(mask_true, axis=-1)
    return K.sqrt(masked_mse)

In [24]:
# Sequential Model

model = Sequential()

model.add(Dense(28, input_dim = X.shape[1], activation='selu'))
model.add(Dense(56, activation='selu'))
model.add(Dense(56, activation='selu'))
model.add(Dropout(0.65))
model.add(Dense(56, activation='selu'))
model.add(Dense(28, activation='selu'))
model.add(Dense(X.shape[1], activation='selu'))

In [25]:
model.compile(loss=rmse, optimizer='sgd')

model.fit(X, X, batch_size=128, epochs=100, validation_data=(X_valid, X_valid), callbacks=[tensorboard])

Train on 13118 samples, validate on 2255 samples
Epoch 1/100
13118/13118 [==============================] - 1s 94us/step - loss: 3.4304 - val_loss: 3.5677
Epoch 2/100
13118/13118 [==============================] - 1s 70us/step - loss: 3.0267 - val_loss: 3.5193
Epoch 3/100
13118/13118 [==============================] - 1s 68us/step - loss: 2.6653 - val_loss: 3.4240
Epoch 4/100
13118/13118 [==============================] - 1s 68us/step - loss: 2.1271 - val_loss: 3.2808
Epoch 5/100
13118/13118 [==============================] - 1s 66us/step - loss: 1.6563 - val_loss: 3.1028
Epoch 6/100
13118/13118 [==============================] - 1s 70us/step - loss: 1.4106 - val_loss: 2.9612
Epoch 7/100
13118/13118 [==============================] - 1s 65us/step - loss: 1.2896 - val_loss: 2.8490
Epoch 8/100
13118/13118 [==============================] - 1s 64us/step - loss: 1.2149 - val_loss: 2.7493
Epoch 9/100
13118/13118 [==============================] - 1s 64us/step - loss: 1.1704 - val_loss: 2.67

13118/13118 [==============================] - 1s 68us/step - loss: 0.8093 - val_loss: 1.6828
Epoch 78/100
13118/13118 [==============================] - 1s 67us/step - loss: 0.8032 - val_loss: 1.6812
Epoch 79/100
13118/13118 [==============================] - 1s 69us/step - loss: 0.8019 - val_loss: 1.6831
Epoch 80/100
13118/13118 [==============================] - 1s 68us/step - loss: 0.8040 - val_loss: 1.6805
Epoch 81/100
13118/13118 [==============================] - 1s 66us/step - loss: 0.8050 - val_loss: 1.6712
Epoch 82/100
13118/13118 [==============================] - 1s 67us/step - loss: 0.8021 - val_loss: 1.6701
Epoch 83/100
13118/13118 [==============================] - 1s 69us/step - loss: 0.8043 - val_loss: 1.6716
Epoch 84/100
13118/13118 [==============================] - 1s 70us/step - loss: 0.7986 - val_loss: 1.6678
Epoch 85/100
13118/13118 [==============================] - 1s 67us/step - loss: 0.7969 - val_loss: 1.6663
Epoch 86/100
13118/13118 [========================

In [26]:
test_loss = model.evaluate(X_test, X_test)
test_loss

2286/2286 [==============================] - 0s 50us/step


1.6149727233334372